# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121264e+02     1.626245e+00
 * time: 0.08834385871887207
     1     1.111821e+01     9.282608e-01
 * time: 0.9482319355010986
     2    -1.220802e+01     9.980154e-01
 * time: 1.2058169841766357
     3    -3.401709e+01     7.476162e-01
 * time: 1.3844289779663086
     4    -4.763729e+01     5.749256e-01
 * time: 1.8203768730163574
     5    -5.674707e+01     2.234180e-01
 * time: 2.1936779022216797
     6    -5.962044e+01     1.622585e-01
 * time: 2.3106119632720947
     7    -6.082845e+01     6.516739e-02
 * time: 2.4193248748779297
     8    -6.134413e+01     5.545606e-02
 * time: 2.5376999378204346
     9    -6.160821e+01     5.769587e-02
 * time: 2.652700901031494
    10    -6.181836e+01     2.652704e-02
 * time: 2.766119956970215
    11    -6.197664e+01     2.363862e-02
 * time: 2.8725669384002686
    12    -6.206335e+01     1.801765e-02
 * time: 2.9855358600616455
    13    -6.211433e+01     1.760823e-02
 * time: 3.09979605674

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557662
    AtomicNonlocal      14.8522636
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336819

    total               -62.261666458583
